In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    

In [4]:
# ModelCheckpoint 콜백 : 예상치 못한 사고로 인해 학습이 마무리되지 못했을 경우를 대비하여 매 에포크마다 모델 파일 생성

### save_best_only 파라미터를 True로 넘겨주어 가장 좋은 결과를 보였던 훈련 회차의 결과만 저장되도록 할 수도 있음.

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb])
model = keras.models.load_model("my_keras_model.h5")
mse_test = model.evaluate(X_test, y_test)

Epoch 1/10
363/363 [==============================] - 3s 6ms/step - loss: 2.3864 - val_loss: 1.1804
Epoch 2/10
363/363 [==============================] - 1s 2ms/step - loss: 0.8160 - val_loss: 0.7182
Epoch 3/10
363/363 [==============================] - 1s 2ms/step - loss: 0.6655 - val_loss: 0.6677
Epoch 4/10
363/363 [==============================] - 1s 2ms/step - loss: 0.6298 - val_loss: 0.6440
Epoch 5/10
363/363 [==============================] - 1s 2ms/step - loss: 0.6079 - val_loss: 0.6270
Epoch 6/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5914 - val_loss: 0.6109
Epoch 7/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5765 - val_loss: 0.5994
Epoch 8/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5650 - val_loss: 0.5863
Epoch 9/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5530 - val_loss: 0.5751
Epoch 10/10
162/162 [==============================] - 0s 1ms/step - loss: 0.5571


In [7]:
# EarlyStopping 콜백 : 일정 에포크동안 검증 세트에 대한 점수가 향상되지 않는 경우 훈련을 조기 종료.

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=200,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb])
mse_test = model.evaluate(X_test, y_test)

Epoch 1/200
363/363 [==============================] - 3s 5ms/step - loss: 0.3160 - val_loss: 0.3397
Epoch 2/200
363/363 [==============================] - 1s 4ms/step - loss: 0.3156 - val_loss: 0.3416
Epoch 3/200
363/363 [==============================] - 1s 3ms/step - loss: 0.3153 - val_loss: 0.3423
Epoch 4/200
363/363 [==============================] - 1s 4ms/step - loss: 0.3161 - val_loss: 0.3429
Epoch 5/200
363/363 [==============================] - 1s 4ms/step - loss: 0.3168 - val_loss: 0.3420
Epoch 6/200
363/363 [==============================] - 1s 3ms/step - loss: 0.3152 - val_loss: 0.3398
Epoch 7/200
363/363 [==============================] - 1s 3ms/step - loss: 0.3135 - val_loss: 0.3394
Epoch 8/200
363/363 [==============================] - 1s 2ms/step - loss: 0.3142 - val_loss: 0.3396
Epoch 9/200
363/363 [==============================] - 1s 2ms/step - loss: 0.3131 - val_loss: 0.3393
Epoch 10/200
363/363 [==============================] - 1s 2ms/step - loss: 0.3130 - val_lo

In [ ]:
# 더 많은 제어를 원할 경우 사용자 정의 콜백을 만들 수도 있음.

class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))